In [52]:
import os
import json

In [ ]:
ROOT = r"X:\tsr\TD-TSR"
image_dir = os.path.join(ROOT, "images")
train_annotations = os.path.join(ROOT, "annotations","train.json")
val_annotations = os.path.join(ROOT, "annotations","val.json")

In [54]:
def visualize_image(image, annots):
    import cv2
    import matplotlib.pyplot as plt
    import matplotlib.patches as patches

    img = cv2.imread(image)
    fig, ax = plt.subplots(1, figsize=(12, 12))
    ax.imshow(img)
    for a in annots:
        x, y, w, h = a['bbox']
        rect = patches.Rectangle((x, y), w, h, linewidth=1, edgecolor='r', facecolor='none')
        ax.add_patch(rect)
    plt.show()

In [ ]:
import json
import os


def convert_to_yolo_format(annotation, image_width, image_height):
    """
    Converts an annotation from COCO format to YOLO format.

    Args:
        annotation (dict): A dictionary containing 'bbox' and 'category_id'.
        image_width (int): Width of the image.
        image_height (int): Height of the image.

    Returns:
        str: A string representation in YOLO format (category x_center y_center width height).
    """
    x, y, w, h = annotation["bbox"]

    # Normalize values
    x_center = (x + w / 2) / image_width
    y_center = (y + h / 2) / image_height
    w /= image_width
    h /= image_height

    return f"{annotation['category_id']-1} {x_center:.6f} {y_center:.6f} {w:.6f} {h:.6f}"


def get_yolo_annotations(val_annotations):
    """
    Converts all annotations of a specific category to YOLO format.

    Args:
        val_annotations (dict): COCO-style annotation dictionary.
        category_id (int): The category ID to filter.

    Returns:
        dict: A mapping of image filenames to YOLO annotations.
    """
    yolo_annotations = {}

    for image_info in val_annotations["images"]:
        image_id = image_info["id"]
        image_width, image_height = image_info["width"], image_info["height"]

        # Filter annotations for the current image
        annots = [
            a
            for a in val_annotations["annotations"]
            if a["image_id"] == image_id
        ]

        # Convert annotations
        yolo_annots = [
            convert_to_yolo_format(a, image_width, image_height) for a in annots
        ]

        if yolo_annots:
            yolo_annotations[image_info["file_name"]] = yolo_annots

    return yolo_annotations


TRAIN_OUTDIR = r"X:\tsr\TD-TSR\labels\train"

# Create output directory if it doesn't exist
os.makedirs(TRAIN_OUTDIR, exist_ok=True)

train_annotations = json.load(open(train_annotations))
train_yolo_annotations = get_yolo_annotations(train_annotations)


# Save annotations to disk
for filename, annots in train_yolo_annotations.items():
    output_filename = os.path.join(TRAIN_OUTDIR, os.path.splitext(filename)[0] + ".txt")
    with open(output_filename, "w") as f:
        f.write("\n".join(annots))

print(f"Saved {len(train_yolo_annotations)} annotations to {TRAIN_OUTDIR}")


VAL_OUTDIR = r"X:\tsr\TD-TSR\labels\val"
os.makedirs(VAL_OUTDIR, exist_ok=True)

val_annotations = json.load(open(val_annotations))
val_yolo_annotations = get_yolo_annotations(val_annotations)

# Save annotations to disk
for filename, annots in val_yolo_annotations.items():
    output_filename = os.path.join(VAL_OUTDIR, os.path.splitext(filename)[0] + ".txt")
    with open(output_filename, "w") as f:
        f.write("\n".join(annots))

id2cat = {cat["id"]-1: cat["name"] for cat in val_annotations["categories"]}
with open(os.path.join(ROOT, "id2label.json"), "w") as f:
    json.dump(id2cat, f)

print(f"Saved {len(val_yolo_annotations)} annotations to {VAL_OUTDIR}")

Saved 504 annotations to X:\tsr\TD-TSR\labels\train
Saved 167 annotations to X:\tsr\TD-TSR\labels\val
